# Retrieval Evaluation

In [1]:
import os
# Setting temp dir to /srv/data directory, as it otherwise fills up the home directory too much
# Just comment out on machines that are not "Goober"
os.environ["TMPDIR"] = "/srv/data/tmp"
os.makedirs("/srv/data/tmp", exist_ok=True)

%pip install haystack-ai
%pip install ragas-haystack
%pip install nltk
%pip install openai
%pip install pandas
%pip install ragas-haystack
%pip install sentence-transformers>=3.0.0
%pip install hf_xet
%pip install ollama-haystack==2.4.2
%pip install tqdm # For Progress Bar

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
from haystack_integrations.components.evaluators.ragas import RagasEvaluator
from haystack.document_stores.in_memory import InMemoryDocumentStore
import importlib
from ragas.metrics import NonLLMContextPrecisionWithReference, NonLLMContextRecall
from tqdm import tqdm
tqdm.pandas()
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(message)s',
    datefmt='%H:%M:%S'
)

In [ ]:
from config.secret import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

os.environ["EMBEDDING_MODEL_NAME"] = "Qwen/Qwen3-Embedding-4B"
os.environ["SENTENCE_TRANSFORMERS_HOME"] = "./model-assets/sentence-transformers"
os.environ["EMBEDDING_MODEL_TOP_K"] = "10"

os.environ["LLM_NAME"] = "gemma3:12b"
os.environ["LLM_CONTEXT_SIZE"] = "8192"

In [4]:
df = pd.read_pickle("data/question_answers_with_documents_flat.pkl")
all_documents = pd.read_pickle("data/all_documents.pkl")["document"].tolist()

In [5]:
print(df.head())

                                            question  \
0  Wie läuft eine äußere Wendung ab und was passi...   
1  Und wie lange dauert dieser Eingriff normalerw...   
2  Ey, wie machen die eigentlich so 'ne äußere We...   
3  How is an external version done and what happe...   
4  Können Sie mir erklären, wie eine äußere Wendu...   

                                          references  \
0  [Eine äußere Wendung findet in der Regel in ei...   
1  [Eine äußere Wendung findet in der Regel in ei...   
2  [Eine äußere Wendung findet in der Regel in ei...   
3  [Eine äußere Wendung findet in der Regel in ei...   
4  [Eine äußere Wendung findet in der Regel in ei...   

                                         groundTruth         source_file  \
0  Eine äußere Wendung findet ambulant in einer K...  Äußere Wendung.md   
1  Eine äußere Wendung findet ambulant in einer K...  Äußere Wendung.md   
2  Eine äußere Wendung findet ambulant in einer K...  Äußere Wendung.md   
3  Eine äußere Wendung

## Setup Retrieval Pipelines

Sets up the basic / hybrid RAG pipelines. These can then be used later in combintation with the different indexing methods from before to evaluate different RAG setups.

In [6]:
base_indexing_store = InMemoryDocumentStore.load_from_disk(f"data/document_stores/{os.environ['EMBEDDING_MODEL_NAME']}/base_indexing_store.json")
context_indexing_store = InMemoryDocumentStore.load_from_disk(f"data/document_stores/{os.environ['EMBEDDING_MODEL_NAME']}/context_indexing_store.json")

In [7]:
import pipelines.retrieval_pipelines.base_retrieval_pipeline
importlib.reload(pipelines.retrieval_pipelines.base_retrieval_pipeline)

base_retrieval_pipeline = pipelines.retrieval_pipelines.base_retrieval_pipeline.get_base_retrieval_pipeline(base_indexing_store)

ragas_evaluator = RagasEvaluator(
    ragas_metrics=[NonLLMContextPrecisionWithReference(), NonLLMContextRecall()]
)

base_retrieval_pipeline.add_component(instance=ragas_evaluator, name="ragas_evaluator")
base_retrieval_pipeline.connect("retriever", "ragas_evaluator.documents")

17:25:35 INFO Load pretrained SentenceTransformer: Qwen/Qwen3-Embedding-4B


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

17:25:41 INFO 2 prompts are loaded, with the keys: ['query', 'document']


🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - ragas_evaluator: RagasEvaluator
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> ragas_evaluator.documents (List[Document])

In [8]:
import pipelines.retrieval_pipelines.hybrid_retrieval_pipeline
importlib.reload(pipelines.retrieval_pipelines.hybrid_retrieval_pipeline)

hybrid_retrieval_pipeline = pipelines.retrieval_pipelines.hybrid_retrieval_pipeline.get_hybrid_retrieval_pipeline(base_indexing_store)

ragas_evaluator = RagasEvaluator(
    ragas_metrics=[NonLLMContextPrecisionWithReference(), NonLLMContextRecall()]
)

hybrid_retrieval_pipeline.add_component(instance=ragas_evaluator, name="ragas_evaluator")
hybrid_retrieval_pipeline.connect("retriever", "ragas_evaluator.documents")

🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - bm25_retriever: InMemoryBM25Retriever
  - retriever: InMemoryEmbeddingRetriever
  - joiner: DocumentJoiner
  - ragas_evaluator: RagasEvaluator
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - bm25_retriever.documents -> joiner.documents (List[Document])
  - retriever.documents -> joiner.documents (List[Document])
  - retriever.documents -> ragas_evaluator.documents (List[Document])

In [9]:
import pipelines.retrieval_pipelines.base_retrieval_pipeline
importlib.reload(pipelines.retrieval_pipelines.base_retrieval_pipeline)

context_retrieval_pipeline = pipelines.retrieval_pipelines.base_retrieval_pipeline.get_base_retrieval_pipeline(context_indexing_store)

ragas_evaluator = RagasEvaluator(
    ragas_metrics=[NonLLMContextPrecisionWithReference(), NonLLMContextRecall()]
)

context_retrieval_pipeline.add_component(instance=ragas_evaluator, name="ragas_evaluator")
context_retrieval_pipeline.connect("retriever", "ragas_evaluator.documents")

🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - ragas_evaluator: RagasEvaluator
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> ragas_evaluator.documents (List[Document])

In [10]:
import pipelines.retrieval_pipelines.hybrid_retrieval_pipeline
importlib.reload(pipelines.retrieval_pipelines.hybrid_retrieval_pipeline)

context_hybrid_retrieval_pipeline = pipelines.retrieval_pipelines.hybrid_retrieval_pipeline.get_hybrid_retrieval_pipeline(context_indexing_store)

ragas_evaluator = RagasEvaluator(
    ragas_metrics=[NonLLMContextPrecisionWithReference(), NonLLMContextRecall()]
)

context_hybrid_retrieval_pipeline.add_component(instance=ragas_evaluator, name="ragas_evaluator")
context_hybrid_retrieval_pipeline.connect("retriever", "ragas_evaluator.documents")

🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - bm25_retriever: InMemoryBM25Retriever
  - retriever: InMemoryEmbeddingRetriever
  - joiner: DocumentJoiner
  - ragas_evaluator: RagasEvaluator
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - bm25_retriever.documents -> joiner.documents (List[Document])
  - retriever.documents -> joiner.documents (List[Document])
  - retriever.documents -> ragas_evaluator.documents (List[Document])

## Run Evaluation

In [11]:
test_cases = [
    {
        "name": "Basic RAG",
        "pipeline": base_retrieval_pipeline,
    },
    {
        "name": "Hybrid RAG",
        "pipeline": hybrid_retrieval_pipeline,
    },
    {
        "name": "Contextual RAG",
        "pipeline": context_retrieval_pipeline,
    },
    {
        "name": "Contextual Hybrid RAG",
        "pipeline": context_hybrid_retrieval_pipeline,
    },
]

In [12]:
import numpy as np
from scipy.stats import norm

def _has_ci_converged(measurements, tol, alpha=0.05):
    """
    Return True if the (1-alpha)% confidence‐interval half‐width
    of the mean is <= tol.
    """
    n = len(measurements)
    if n < 2:
        return False
    mean = np.mean(measurements)
    var  = np.var(measurements, ddof=1)
    se = np.sqrt(var / n)
    z = norm.ppf(1 - alpha/2)
    half_width = z * se
    logging.info(
        "n=%d, mean=%.4f, var=%.6f, SE=%.6f, z=%.3f → half_width=%.6f (tol=%.6f)",
        n, mean, var, se, z, half_width, tol
    )
    return half_width <= tol

def did_converge(precisions, recalls, tolerance=0.01, alpha=0.05):
    """
    Decide whether to stop based on the CI half-width
    of both precision and recall.

    :param precisions: List[float]
    :param recalls:    List[float]
    :param step_size:  (unused) kept for API compatibility
    :param tolerance:  float, max allowable CI half-width
    :param alpha:      float, significance level (default 0.05 ⇒ 95% CI)
    :return:           bool, True if both metrics have CI half-width <= tolerance
    """
    return (_has_ci_converged(precisions, tolerance, alpha) and
            _has_ci_converged(recalls,    tolerance, alpha))

In [13]:
import logging

# silence haystack’s pipeline logs
logging.getLogger("haystack").setLevel(logging.WARNING)
logging.getLogger("haystack.core.pipeline").setLevel(logging.WARNING)

# if you see similar spam from transformers, ragas, etc.
logging.getLogger("transformers").setLevel(logging.WARNING)
logging.getLogger("ragas").setLevel(logging.WARNING)

In [14]:
import prompt
importlib.reload(prompt)

results = []

df["context_precision"] = None
df["context_recall"] = None

adaptive_eval_step_size = 5
number_of_questions_in_analysis = 50

precisions = {tc["name"]: [] for tc in test_cases}
recalls    = {tc["name"]: [] for tc in test_cases}

# 1) Filter out the null‐question rows
df_nonnull = df[df["question"].notnull()]

df_shuffled = df_nonnull.sample(n=50, random_state=42).reset_index(drop=True)

# 3) Now iterate

for index, row in tqdm(
    df_shuffled.iterrows(),
    total=len(df_shuffled),
    desc="Processing rows",
    unit="row"
):
    if index % adaptive_eval_step_size == 0 and index != 0:
        batch_num = index // adaptive_eval_step_size
        logging.info(f"Batch {batch_num}: checking convergence at n={index}")
        statuses = {}
        for name in precisions:
            converged = did_converge(
                precisions[name],
                recalls[name],
            )
            statuses[name] = converged
            logging.info(f"  • {name}: {'✔ converged' if converged else '✘ not yet'}")

        if all(statuses.values()):
            logging.info(f"All test cases converged at n={index}. Stopping early.")
            break
    reference = row["groundTruth"]
    relevant_context = [document.content for document in row["documents"]]
    question = row["question"]

    for test_case in test_cases:
        pipeline = test_case["pipeline"]
        request_payload = {
            "query_embedder": {
                "text": question,
            },
            "ragas_evaluator": {
                "query": question,
                "reference": reference,
                "reference_contexts": relevant_context,
            }
        }
        if "bm25_retriever" in pipeline.graph.nodes:
            request_payload["bm25_retriever"] = {
                "query": question
            }
        result = pipeline.run(request_payload)

        context_precision = result.get("ragas_evaluator", {}).get("result", {})["non_llm_context_precision_with_reference"]
        context_recall = result.get("ragas_evaluator", {}).get("result", {})["non_llm_context_recall"]
        df.at[index, f"{test_case['name']}_precision"] = context_precision
        df.at[index, f"{test_case['name']}_recall"] = context_recall

        # append for convergence checking
        precisions[test_case["name"]].append(context_precision)
        recalls[test_case["name"]].append(context_recall)

Processing rows:   0%|          | 0/50 [00:00<?, ?row/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Processing rows: 100%|██████████| 50/50 [02:47<00:00,  3.34s/row]


In [17]:
embedding_model_name = os.environ['EMBEDDING_MODEL_NAME'].replace('/', '')
df.to_pickle(f"results/retrieval_eval_results_{embedding_model_name}_{os.environ['EMBEDDING_MODEL_TOP_K']}_n{number_of_questions_in_analysis}_topk{os.environ['EMBEDDING_MODEL_TOP_K']}.pkl")

In [16]:
df

,question,references,groundTruth,source_file,documents,variant,prev_messages,context_precision,context_recall,Basic RAG_precision,Basic RAG_recall,Hybrid RAG_precision,Hybrid RAG_recall,Contextual RAG_precision,Contextual RAG_recall,Contextual Hybrid RAG_precision,Contextual Hybrid RAG_recall
0,Wie läuft eine äußere Wendung ab und was passi...,[Eine äußere Wendung findet in der Regel in ei...,Eine äußere Wendung findet ambulant in einer K...,Äußere Wendung.md,[Document(id=ea0c518b7c51e474b09770e8db9709652...,default,[],None,None,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000,0.0
1,Und wie lange dauert dieser Eingriff normalerw...,[Eine äußere Wendung findet in der Regel in ei...,Eine äußere Wendung findet ambulant in einer K...,Äußere Wendung.md,[Document(id=ea0c518b7c51e474b09770e8db9709652...,contextual,"[Wie läuft eine äußere Wendung ab?, Bei der äu...",None,None,0.833333,1.0,0.833333,1.0,0.625,1.0,0.625,1.0
2,"Ey, wie machen die eigentlich so 'ne äußere We...",[Eine äußere Wendung findet in der Regel in ei...,Eine äußere Wendung findet ambulant in einer K...,Äußere Wendung.md,[Document(id=ea0c518b7c51e474b09770e8db9709652...,slang,[],None,None,1.000000,1.0,1.000000,1.0,0.000,0.0,0.000,0.0
3,How is an external version done and what happe...,[Eine äußere Wendung findet in der Regel in ei...,Eine äußere Wendung findet ambulant in einer K...,Äußere Wendung.md,[Document(id=ea0c518b7c51e474b09770e8db9709652...,english,[],None,None,0.500000,1.0,0.500000,1.0,0.000,0.0,0.000,0.0
4,"Können Sie mir erklären, wie eine äußere Wendu...",[Eine äußere Wendung findet in der Regel in ei...,Eine äußere Wendung findet ambulant in einer K...,Äußere Wendung.md,[Document(id=ea0c518b7c51e474b09770e8db9709652...,german-variation-1,[],None,None,1.000000,1.0,1.000000,1.0,1.000,1.0,1.000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,"Was ist zu tun, wenn nach der Geburtseinleitun...",[Bei jeder medikamentösen oder mechanischen Me...,Sollten nach der Geburtseinleitung starke oder...,Einleitung der Geburt.md,[Document(id=8a64407f7a87fa95d3ce2f268e6b00c66...,german-variation-2,[],None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
606,"Kann mir jemand sagen, was ich machen soll, we...",[Bei jeder medikamentösen oder mechanischen Me...,Sollten nach der Geburtseinleitung starke oder...,Einleitung der Geburt.md,[Document(id=8a64407f7a87fa95d3ce2f268e6b00c66...,german-variation-3,[],None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,Ich hatte eine Geburtseinleitung – was mache i...,[Bei jeder medikamentösen oder mechanischen Me...,Sollten nach der Geburtseinleitung starke oder...,Einleitung der Geburt.md,[Document(id=8a64407f7a87fa95d3ce2f268e6b00c66...,german-variation-4,[],None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,"Was empfehlen Sie, wenn nach der Einleitung de...",[Bei jeder medikamentösen oder mechanischen Me...,Sollten nach der Geburtseinleitung starke oder...,Einleitung der Geburt.md,[Document(id=8a64407f7a87fa95d3ce2f268e6b00c66...,german-variation-5,[],None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
